# LangChain Agents Intro
LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Agents, adding the ability to use tools such as search and calculators to complete tasks that normal LLMs cannot fufil. In this example we will be using Ollama llama3.2


In [21]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [22]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x0000020218179620>)

In [23]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [24]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [25]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [26]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [27]:
exponentiate.func(**llm_output_dict)

25

# Creating an Agent
We're going to construct a simple tool calling agent. We will use LangChain Epression Language (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:

agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
We need this agent to remember previous interactions within the conversation. To do that, we will use the ChatPromptTemplate with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's "notes" as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"), #scratchpad is where we store our agent's internal thoughts and thinking steps
])

In [29]:
import os
from langchain_ollama import ChatOllama

# For normal accurate responses
llm = ChatOllama(temperature=0.0, model="llama3.2")

In [30]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

Now we will initialize our agent. For that we need:

- llm: as already defined
- tools: to be defined (just a list of our previously defined tools)
- prompt: as already defined
- memory: as already defined

In [31]:
from langchain.agents import create_tool_calling_agent
tools = [add, subtract, multiply, exponentiate]
agent = create_tool_calling_agent(
    llm = llm, tools=tools, prompt=prompt
)

In [32]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'x': '10.7', 'y': '7.68'}, log="\nInvoking: `multiply` with `{'x': '10.7', 'y': '7.68'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-14T10:41:50.4477466Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1460438300, 'load_duration': 38006700, 'prompt_eval_count': 356, 'prompt_eval_duration': 430733200, 'eval_count': 26, 'eval_duration': 988889900, 'model_name': 'llama3.2'}, id='run--0924bd8e-47ac-40dd-af93-ba5e18bf8dd8-0', tool_calls=[{'name': 'multiply', 'args': {'x': '10.7', 'y': '7.68'}, 'id': '352927cc-a3c0-4c11-a135-1f0ca0f74613', 'type': 'tool_call'}], usage_metadata={'input_tokens': 356, 'output_tokens': 26, 'total_tokens': 382})], tool_call_id='352927cc-a3c0-4c11-a135-1f0ca0f74613')]

In [33]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [34]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': '10.7', 'y': '7.68'}`


82.17599999999999The result of multiplying 10.7 by 7.68 is approximately 82.18.

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 10.7 by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={})],
 'output': 'The result of multiplying 10.7 by 7.68 is approximately 82.18.'}

In [35]:
10.7*7.68

82.17599999999999

In [36]:
agent_executor.invoke({
    "input": "My name is James",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.17599999999999James, the result of multiplying 10.7 by 7.68 is approximately 82.18.

> Finished chain.


{'input': 'My name is James',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 10.7 by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='James, the result of multiplying 10.7 by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={})],
 'output': 'James, the result of multiplying 10.7 by 7.68 is approximately 82.18.'}

In [37]:
agent_executor.invoke({
    "input": "What is 9+10-4*2**3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': '4', 'y': '3'}`


12.0
Invoking: `exponentiate` with `{'x': '9', 'y': '1'}`


9.0
Invoking: `add` with `{'x': '10', 'y': '6'}`


16.0
Invoking: `subtract` with `{'x': '14', 'y': '4'}`


-10.0The result of the expression 9+10-4*2**3 is -10.

> Finished chain.


{'input': 'What is 9+10-4*2**3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 10.7 by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='James, the result of multiplying 10.7 by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is 9+10-4*2**3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of the expression 9+10-4*2**3 is -10.', additional_kwargs={}, response_metadata={})],
 'output': 'The result of the expression 9+10-4*2**3 is -10.'}

In [38]:
 9+10-4*2**3

-13

In [39]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `subtract` with `{'x': '9', 'y': '10'}`


1.0
Invoking: `multiply` with `{'x': '4', 'y': '2'}`


8.0
Invoking: `exponentiate` with `{'x': '2', 'y': '3'}`


8.0
Invoking: `add` with `{'x': '0', 'y': '-10'}`


-10.0Your name is James, and the result of the expression 9+10-4*2**3 is -10.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 10.7 by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='James, the result of multiplying 10.7 by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is 9+10-4*2**3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of the expression 9+10-4*2**3 is -10.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is James, and the result of the expression 9+10-4*2**3 is -10.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is James, and the result of the exp

# SerpAPI Weather Agent
In this example, we'll be using the same agent and executor setup as before, but we'll be adding the SerpAPI service to allow our agent to search the web for information.

To use this tool, you need an API key, with the free plan you can use up to 100 searches per mont

In [ ]:
from getpass import getpass
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") \
    or getpass("Enter your SerpAPI API key: ")

In [95]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

In [96]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [97]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [98]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

In [99]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2025-05-14 15:34:51
Invoking: `Search` with `weather today`




ValueError: Got error from SerpAPI: Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key

In [ ]:
"95f4af7e8cf97a1d65f938b1926eef23"


In [ ]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))